Enable the `autoreload` extension, which will automatically reload the `playfab_manager` module when the source code is updated, using the following magic commands:

In [1]:
%load_ext autoreload
%autoreload 2

Import the `PlayFabManager` class from the `playfab_manager` module and create a `PlayFabManager` instance

In [2]:
from playfab_manager.playfab_manager import PlayFabManager
playfab = PlayFabManager()

Entity token is created


Use the `get_all_players` method of the `PlayFabManager` instance to retrieve a list of all players from the PlayFab:

In [3]:
# NOTE: players are already sorted by last login time
playfab.get_all_players()

Found 191 players


## Get traces

First, select recent players and get data for them

In [4]:
# Import the datetime module
import datetime

# Get the current time
current_time = datetime.datetime.now(datetime.timezone.utc)

# select the players in the last 24 hours
current_players = [p for p in playfab.all_players if current_time - p.LastLogin <= datetime.timedelta(hours=24)]

# print the last three player ids
[p.PlayerId for p in current_players[:4]]

['22BDA0E2269641A9',
 'AC6A87E6506E72EE',
 'E330FC850A800955',
 '6F9EBD54BAFFBFCC']

Second, get files for each of the player

In [5]:
ids = [p.PlayerId for p in current_players[:3]]
playfab.download_player_files(ids)

Found 3 files for player 22BDA0E2269641A9
Found 3 files for player 22BDA0E2269641A9
Found 2 files for player AC6A87E6506E72EE
Found 2 files for player AC6A87E6506E72EE
Found 2 files for player E330FC850A800955
Found 2 files for player E330FC850A800955


Merged files are stored in `TracesPandas`

In [6]:
playfab.all_players[0].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,29999,0,0,0,0,0
1,30099,0,0,351,0,0
2,30199,0,0,341,0,0
3,30299,0,0,331,0,0
4,30399,0,0,321,0,0


In [7]:
playfab.all_players[1].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,31079,0,-125,0,0,0
1,31179,0,-125,0,0,0
2,31279,0,-125,0,0,0
3,31379,0,-125,0,0,0
4,31479,0,-125,0,0,0


In [8]:
playfab.all_players[2].TracesPandas.head()

,timestamp,x,z,rotation,signaling,score
0,32619,120,0,0,0,0
1,32719,120,0,0,0,0
2,32819,120,0,0,0,0
3,32919,120,0,0,0,0
4,33019,120,0,0,0,0


In [73]:
import pandas as pd
import numpy as np
from visualization.animate_traces import animate_traces

In [59]:
df1 = playfab.all_players[0].TracesPandas
df1['time'] = round(df1["timestamp"] / 1000, 1)
df1['pos1'] = [(x, z) for (x, z) in zip(df1.x,  df1.z)]
df2 = playfab.all_players[1].TracesPandas
df2['time'] = round(df2["timestamp"] / 1000, 1)
df2['pos2'] = [(x, z) for (x, z) in zip(df2.x,  df2.z)]
df3 = playfab.all_players[2].TracesPandas
df3['time'] = round(df3["timestamp"] / 1000, 1)
df3['pos3'] = [(x, z) for (x, z) in zip(df3.x,  df3.z)]

In [60]:
# tolerance=pd.Timedelta("2D")

df = pd.merge_asof(df1.loc[:,['time', 'pos1']],
                   df2.loc[:,['time', 'pos2']],
                   on="time")

df = pd.merge_asof(df,
                   df3.loc[:,['time', 'pos3']],
                   on="time")


In [96]:
df.head()

,time,pos1,pos2,pos3
0,207.0,"(0, -125)",NaN,NaN
1,207.1,"(0, -125)",NaN,NaN
2,207.2,"(0, -125)",NaN,NaN
3,207.3,"(0, -125)",NaN,NaN
4,207.4,"(0, -125)",NaN,NaN


In [98]:
filename = "_".join([i for i in ids]) + "_traces.csv"
df.to_csv(filename, index=False)

In [95]:
df.fillna(method="bfill")

,time,pos1,pos2,pos3
0,207.0,"(0, -125)","(120, 0)","(0, 120)"
1,207.1,"(0, -125)","(120, 0)","(0, 120)"
2,207.2,"(0, -125)","(120, 0)","(0, 120)"
3,207.3,"(0, -125)","(120, 0)","(0, 120)"
4,207.4,"(0, -125)","(120, 0)","(0, 120)"
...,...,...,...,...
1388,345.8,"(-3, -58)","(111, 2)","(5, 32)"
1389,345.9,"(-3, -58)","(111, 2)","(5, 32)"
1390,346.0,"(-3, -58)","(111, 2)","(5, 32)"
1391,346.1,"(-3, -58)","(111, 2)","(5, 32)"


In [97]:
filename = "_".join([i for i in ids]) + "_traces.gif"
df_filled = df.fillna(method="bfill")

valid_times = np.logical_and(df_filled.notna()["pos1"], df_filled.notna()["pos2"])
valid_times = np.logical_and(valid_times, df_filled.notna()["pos3"])

animate_traces(df_filled.loc[valid_times, :][:1000], save=True, filename=filename)

In [90]:
df_filled.loc[valid_times, :].shape[0]

1163